In [1]:
import os

os.chdir("../")
%pwd

'/Users/valentinmonney/Documents/data_science/python/MachineLearningProjects/Chest-X-Ray-Pathology-Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [3]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
        )

        return prepare_base_model_config

In [4]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

2023-10-11 16:49:44.084412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
from tensorflow.keras import layers, Model


class PrepareBaseModel:
    """Prepare the base model for training."""

    def __init__(self, config: PrepareBaseModelConfig) -> None:
        """Initialize the class."""
        self.config = config

    def get_base_model(self) -> None:
        """Retrieve the base InceptionV3 model and save it."""
        self.model = tf.keras.applications.inception_v3.InceptionV3(
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(
        model: tf.keras.Model,
        classes: int,
        freeze_all: bool,
        learning_rate: float,
        freeze_till: str,
    ) -> tf.keras.Model:
        """Prepare the full model for training."""
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till:
            for layer in model.layers:
                layer.trainable = False
            for layer in model.get_layer(freeze_till).output:
                layer.trainable = True

        last_layer = model.get_layer(freeze_till)
        last_output = last_layer.output

        # Additional architecture as per the provided description
        x = layers.Flatten()(last_output)
        x = layers.Dense(1024, activation="relu")(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(classes, activation="sigmoid")(x)

        full_model = Model(model.input, x)
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss="binary_crossentropy",
            metrics=["accuracy"],
        )
        full_model.summary()
        return full_model

    def update_base_model(self) -> None:
        """Update the base model with the new number of classes."""
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till="mixed7",
            learning_rate=self.config.params_learning_rate,
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model) -> None:
        """Save the model."""
        model.save(path)

In [8]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-10-11 16:52:02,794: INFO: common] yaml file: config/config.yaml loaded successfully
[2023-10-11 16:52:02,799: INFO: common] yaml file: params.yaml loaded successfully
[2023-10-11 16:52:02,800: INFO: common] created directory at: artifacts
[2023-10-11 16:52:02,801: INFO: common] created directory at: artifacts/prepare_base_model
[2023-10-11 16:52:04,135: WARNING: saving_utils] Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                   